# Recuperación de la Información y Minería de Texto
# Análisis de noticias

Práctica desarrollada por :

    Cesar González Fernández [cesar.gon.fer@gmail.es]
    Carlos Correa García [carlos.correa.88@gmail.es]

## Índice

1. [Introducción](#introduction)
2. [Casos analizados y ejecutados](#cases)
3. [Conclusiones](#conclusions)

<a id='introduction'></a>
# 1. Introducción

A lo largo de esta práctica, vamos a modelizar y testear diferentes representaciones sobre diferentes noticias recogidas de algunos de los periódicos más influyentes de hoy en día. Esto es, analizaremos diferentes formas de representar un texto como un conjunto de términos que identifiquen a los documentos. De esta forma, al aplicar un algoritmo de clustering sobre esos conjuntos de términos, clasificaremos las noticias mediante la similitud entre términos y obtendremos un resultado de pertenencia a determinados clusters de los diferentes documentos.

Una vez obtenida esa agrupación, mediremos la potencia de la solución mediante la diferencia coseno con el vector de clúster real, donde se representa a qué cluster deben pertenecer las noticias por su temática.

Para ello, utilizaremos dos de las bibliotecas para Python más conocidas de NLP (Natural Language Processing):

* **NLTK** 
* **Spacy**

<div class  = "alert alert-info"> 
Si desea más información de NLTK y Spacy, es posible visitar los siguientes enlaces: [
NLTK: https://www.nltk.org/]
[Spacy: https://spacy.io/]
</div>

Incluimos las bibliotecas y referencias que van a ser necesarias, entre las que destacan además de nltk y spacy, numpy y sklearn.

In [1]:
from time import time
import os
import pprint
import re

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import *
from sklearn.metrics.cluster import adjusted_rand_score
import nltk
import numpy
import spacy

Así como algunas funciones programadas por nosotros y que se explicarán en el anexo.

In [2]:
from practica import *
from utils.parser import html2txt_parser_dir

Para comparar el vector resultado de nuestras pruebas con el resultado real esperado, creamos la variable REFERENCE con la información de clusters. Cada elemento de esta lista corresponde con el identificador de cluster al que pertenece. Si nos paramos a echar un vistazo a las noticias, veremos como efectivamente, la noticia 1, 3, 4, 5 y 20 tratan sobre el mismo tema. De igual manera la noticia 2, 10, 11, 12, 13, 14 y 22, y así sucesivamente.
También se ha creado un vector de idiomas si más razón de ser que la de poder imprimirlo junto con los resultados y poder ver la influencia de estos más claramente.

In [ ]:
REFERENCE = [0, 5, 0, 0, 0, 2, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 0, 2, 5]
LANG_REF = ['E', 'E', 'E', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'E', 'E', 'E',    
            'S', 'E', 'E', 'S', 'E', 'E', 'E', 'E', 'S'] 

Además, nos hará falta cargar una variable con las utilidades y diccionario de Spacy en el idioma inglés:

In [ ]:
nlp = spacy.load('en')

<a id='cases'></a>
# 2. Casos analizados y ejecutados

Hemos analizado diferentes casos, aplicando sobre el texto diferentes transformaciones y analizando resultados, y así ir aplicando sobre el texto las métricas que mejor se comportan en reconocimiento de texto. Comenzamos con el más sencillo, como es simplemente tokenizar, y terminaremos con la obtención de entidades nombradas y nombres más comunes para obtener su frecuencia booleana.

## 2.1 Tokenización

En primer lugar vamos a empezar por lo más facil, el texto sin trarar. Únicamente necesitamos tokenizar el texto tal cual lo tenemos. Sobre estos tokens, la función de clusterización será capaz de calcular el clustter de cada texto.
Al ser esta el primer caso y el más sencillo, podemos ver claramente en que consisten estás funciones, a los que llamamos casos, en los que hemos dividido el código para hacerlo más fácil de seguir. Los casos esperan recibir como parámetro un fichero, el cuál procesan haciendo uso de funciones externas (consiguiendo así reutilizar código entre diferentes casos) y devolveran una lista con los tokens del texto ya procesado.

In [ ]:
def word_tokenize(in_):
    s = in_ if isinstance(in_, str) else in_.read()
    return nltk.word_tokenize(s)

In [ ]:
@register_case
def case_1(f_):
    """ Ninguna transformación. """

    return word_tokenize(f_)

In [ ]:
print('Score: ', evaluate(case_1))

Como vemos, con esta primera aproximación la puntuación obtenida es muy baja. Esto era de esperar ya que estamos usando todas las palabras del texto, las cuales incluyen las más comunes de ambos lenguajes (como determinantes y preposiciones) además de no traducir los textos a un lenguaje en común, lo que hace muy dificil que coincidan palabras de textos de idimas diferentes a pesar de tratar el mismo tema.

## 2.2 Transformación de tokens a minúsculas

Por lo que extraemos del ejemplo anterior, la tokenización es un método que, sin combinarlo con otros, es bastante malo, puesto que incluso si se obtiene la misma palabra en dos documentos pero difiere una letra minúscula de una mayúscula, el algoritmo de clusterización no va a encontrar similaridad entre ambas palabras, por lo que el ruido entre documentos es mayor.

Es por eso por lo que "normalizamos" todos los tokens a minúscula. De esta forma, obtendremos muy probablemente un número menor de palabras diferentes y por lo tanto, unos clústers un poco mejor definidos.

#CODIGO

In [ ]:
def to_lower_case(tokens):
    return [t.lower() for t in tokens]

In [ ]:
@register_case
def case_2(f_):
    """ Convierte los tokens a minusculas. """

    tokens = word_tokenize(f_)
    return to_lower_case(tokens)

In [ ]:
print('Score: ', evaluate(case_2))

Aún habiendo aplicado la transformación a minúsculas, vemos que el Score es muy bajo, por lo que debemos aplicar métricas para obtener similaridades entre documentos algo más complejas.

## 2.3 Eliminación de signos de puntuación (aplicando el punto anterior)

Los signos de puntuación no añaden significado al texto y son usados en todos, tanto en español como en inglés. Además, el número de repeticiones de estos caracteres es muy grande. Esto evidentemente provoca que otras palabras más significativas pierdan peso

In [ ]:
def remove_puntuation(tokens):
    import string

    return [t for t in tokens if t not in string.punctuation]

In [ ]:
@register_case
def case_3(f_):
    """ Convierte a minusculas y elimina signos de puntuacion. """

    tokens = word_tokenize(f_)
    tokens = to_lower_case(tokens)
    return remove_puntuation(tokens)

In [ ]:
print('Score: ', evaluate(case_3))

Podemos ver que la puntuación tras eliminar los signos de puntuación a aumentado, pero aún está lejos de ser un valor aceptable. Esta mejora es debida a el mayor peso que se da a otros tokens más significativos.

## 2.4 Eliminación de stopwords en inglés y español (aplicando el punto anterior)

Una vez eliminados los signos de puntuación, que no aportan ningún valor añadido a nuestro análisis de textos, nos puede venir a la mente la idea de eliminar más palabras de nuestros textos que no aportan ningún significado extra al documento y que además, se repiten constantemente en diferentes frases; son las que se denominan "stopwords". Dentro de este conjunto de palabras se encuentran las conjunciones, preposiciones, algunos verbos auxiliares, etc.

Para ello, utilizaremos la lista de palabras que nos proporciona el corpus stopwords de la biblioteca **nltk**:

Estas son las stopwords que eliminaremos del inglés:

In [ ]:
print (nltk.corpus.stopwords.words('english'))

Y las del español:

In [ ]:
print (nltk.corpus.stopwords.words('spanish'))


In [ ]:
def remove_stop_words(tokens, langs = ['english']):
    return [t for t in tokens if t not in nltk.corpus.stopwords.words(langs)]

In [ ]:
@register_case
def case_4(f_):
    """ Lo anterior y elimina las stopwords es español e ingles. """

    tokens = word_tokenize(f_)
    tokens = to_lower_case(tokens)
    tokens = remove_puntuation(tokens)
    return remove_stop_words(tokens, langs=['english', 'spanish'])

In [ ]:
print('Score: ', evaluate(case_4))

Tras eliminar todas las stopwords, vemos que el Score obtenido es mayor a ninguna de las anteriores ejecuciones, por lo que mantendremos la tokenización y la eliminación de signos de puntuación y stopwords a lo largo de nuestro análisis de textos.

## 2.5 Traducción con TextBlob (aplicando el punto anterior)

Como hemos podido ver en los casos anteriores, los clusteres siempre se han agrupado dando prioridad en primer lugar al idioma y después al tema. Esto es lógico, es complicado que entre dos textos en idiomas diferentes existan más terminos coincidentes que entre dos de idiomas diferentes. Estos terminos se ven limitados a poco más que nombres propios, fechas o palabras que compartan ambos idiomas.
Es por esto que el siguiente paso lógico es el traducir todos los textos al mismo idioma. Ya que la mayor parte de los textos están en inglés, y que los paquetes con los que trabajamos son más potentes cuando trabajan con este idioma, el sentido de la traducción que usaremos será de español a inglés:

In [ ]:
def translate_text(f_, method='textblob', target='en'):
    from textblob import TextBlob

    s = f_.read()
    tb = TextBlob(s)
    if TextBlob(s).detect_language() == target:
        return s
    if method == 'textblob':
        return str(tb.translate(to=target))
    else:
        return translate_deepl(s)

In [ ]:
@register_case
def case_5(f_):
    """ Traducido con TextBlob. """

    text = translate_text(f_)
    tokens = word_tokenize(text)
    tokens = to_lower_case(tokens)
    tokens = remove_puntuation(tokens)
    return remove_stop_words(tokens)

In [ ]:
print('Score: ', evaluate(case_5))

Como vemos, la mejora es más que evidente. Además, textos como el 9 y el 11 han sido asignados al mismo cluster a pesar de tener idimas diferentes.
Podemos probar que ocurriría si el idima al que transformamos los textos fuera al español para comprobar si, escoger inglés como idioma con el que trabajar, ha sido buena idea.

In [ ]:
@register_case                                                                  
def case_51(f_):                                                                
    """ Traducido con TextBlob (a español). """                                 
                                                                                  
    text = translate_text(f_, target='es')                                     
    tokens = word_tokenize(text)                                               
    tokens = to_lower_case(tokens)                                             
    tokens = remove_puntuation(tokens)                                         
    return remove_stop_words(tokens, langs=['spanish'])

In [ ]:
print('Score: ', evaluate(case_51))

Como vemos, la diferencia de puntuación es grande, por lo que estabamos en lo cierto al pensar que traducir los textos al inglés era la mejor opción.

## 2.6 Traducción con deepl  (aplicando el punto 2.4)

Al igual que en el caso anterior, vamos a utilizar en este caso la bibiolteca **deepl** y analizar si obtenemos unos clusters más definidos en función de los términos obtenidos mediante traducción a inglés con deepl.

In [ ]:
def translate_deepl(s, target='EN'):
    import deepl

    return '\n'.join([deepl.translate(ss, target=target)[0]
                      for ss in s.split('.')])

In [ ]:
@register_case
def case_6(f_):
    """ Traducido con deepl. """

    text = translate_text(f_, method='deepl')
    tokens = word_tokenize(text)
    tokens = to_lower_case(tokens)
    tokens = remove_puntuation(tokens)
    return remove_stop_words(tokens)

In [ ]:
print('Score: ', evaluate(case_6))

Con la única diferencia de la utilización de una biblioteca u otra de traducción, vemos que se obtiene un Score igual en el caso de utilizar deepl. 

Vamos a utilizar TextBlob para traducir en los siguientes puntos por 2 razones:

    1) En pruebas anteriores, obtuvimos un Score menor con deepl frente al Score de TextBlob. Eso nos hace pensar que la traducción online realizada por deepl ha sido mejorada desde entonces, pero en el mejor de los casos, obtenemos el mismo resultado que con TextBlob, por lo que descartaremos deepl.
    
    2) La traducción con TextBlob es mucho más liviana y su tiempo de ejecución es considerablemente menor.




## 2.7 Eliminación de stopwords ampliadas

Las stopwords utilizadas en los casos anterior son términos definidos por las librerias y se basan en el estudio de los idiomas para poder ser usadas en todos los casos. 
Estas stopwords son útiles para nuestro corpus de documentos, pero podemos hacer una aproximación mas específica para nuestros datos. Ya que el objetivo de las stopwords es identificar aquellas palabras más comunes y que por tanto, dejan de ser útiles a la hora de clasificar, vamos a extraer de nuestros documentos aquellos términos que aparecen con más frecuencia.

In [ ]:
def remove_expanded_stop_words(tokens):
    expanded_stopwords =  nltk.corpus.stopwords.words('english') + ['the',
            'say', '-PRON-', 'people', 'year', 'take','international',
            'state', 'new', 'try', 'report', 'leader','government', 'tell',
            'minister', 'leave','support', 'region', 'work', 'want', 'call',
            'continue','in', 'time', 'week', 'member', 'need', 'policy',
            'news','country', 'later', 'receive', 'force', 'face', 'public',
            'sign']
    return [t for t in tokens if t not in expanded_stopwords]

In [ ]:
@register_case
def case_7(f_):
    """ Stopwords ampliadas para el corpus. """

    text = translate_text(f_)
    tokens = word_tokenize(text)
    tokens = to_lower_case(tokens)
    tokens = remove_puntuation(tokens)
    return remove_expanded_stop_words(tokens)

In [ ]:
print('Score: ', evaluate(case_7))

## 2.8 Utilización de entidades nombradas

Es natural pensar que existan ciertas palabras singulares en el texto con una importancia muy alta. Por ello, la identificación de entidades nombradas es un método común en procesamiento de texto. Ese método reconoce personas relevantes en el texto, organizaciones, lugares, expresiones de tiempo, cantidades, etc.

Por ello, vamos a hacer uso de la biblioteca spacy y su reconocimiento de entidades nombradas, mediante el atributo "ents".

#CODIGO

In [ ]:
def get_named_entities(text, languague='en'):
    import spacy

    nlp = spacy.load(languague)
    return nlp(text).ents

In [ ]:
@register_case
def case_8(f_):
    """ Con entidades nombradas. """

    text = translate_text(f_)
    return [str(w) for w in get_named_entities(text)]

In [ ]:
print('Score: ', evaluate(case_8))

En este caso, el Score sube hasta 0.815913688469319, reduciendo a 4 las noticias erróneamente clasificadas.

## 2.9 Utilización de entidades nombradas preseleccionadas

In [ ]:
@register_case
def case_9(f_):
    """ Con entidades nombradas filtradas """

    text = translate_text(f_)
    return [str(w) for w in get_named_entities(text) if w.label_ \
            in ['GPE', 'PERSON', 'NORP', 'ORG']]

In [ ]:
print('Score: ', evaluate(case_9))

## 2.10 Utilización de entidades nombradas preseleccionadas y 5 tokens más comunes por texto

Si analizamos cualquier texto o noticia, podemos ver que además de ciertas entidades nombradas interesantes, hay otras palabras comunes que son de mucho interés y aportan gran cantidad de información. Por ejemplo, en noticias sobre el escándalo de Oxfam, podría interesarnos sustantivos (nombres comunes) como "abuso", "sexo" o "verguenza". Esas palabras tienen un peso muy alto en la noticia y que conjuntamente con las entidades nombradas, nos pueden ofrecer un conjunto de términos mucho más completo.

Por ello, vamos a contar el número de sustantivos que aparecen en el texto y a quedarnos con los 5 más significativos.

#CODIGO

In [ ]:
def lemmatizer(text, expanded_stopwords=[], pos=[]):
    return [w.lemma_ for w in nlp(text) if (not (w.is_stop or str(w) in \
                expanded_stopwords or w.is_punct)) and \
                (w.pos_ in pos)]

In [ ]:
@register_case
def case_10(f_):
    """ Con entidades nombradas filtradas y mas comunes. """
    from collections import Counter

    expanded_stopwords =  ['the',
            'say', '-PRON-', '', 'people', 'year', 'take','international',
            'state', 'new', 'try', 'report', 'leader','government', 'tell',
            'minister', 'leave','support', 'region', 'work', 'want', 'call',
            'continue','in', 'time', 'week', 'member', 'need', 'policy',
            'news','country', 'later', 'receive', 'force', 'face', 'public',
            'sign']


    text = translate_text(f_)
    tokens = lemmatizer(text, expanded_stopwords=expanded_stopwords,
                        pos=['NOUN'])
    most_common_nouns = [c[0] for c in Counter(tokens).most_common(5)]
    return to_lower_case([str(w) for w in get_named_entities(text) \
            if w.label_ in ['GPE', 'PERSON', 'NORP', 'ORG', 'DATE']] + \
            most_common_nouns)

In [ ]:
print('Score: ', evaluate(case_10))

Observamos en los resultados de la ejecución que el Score sigue siendo el mismo y sigue existiendo el error en la clusterización que aún teníamos en el caso anterior. Parece que el número de entidades nombradas es muy superior a los 5 sustantivos más comunes de texto.

## 2.11 Utilización de entidades nombradas preseleccionadas y 5 tokens más comunes por texto mediante frecuencia booleana

Puesto que el caso anterior, aunque el Score sea el mismo y el error en la clusterización siga vigente, la información que se aporta de cada noticia es claramente mayor que sin tener en cuenta los nombres más repetidos en el texto. El problema es que las entidades nombradas pueden estar siendo repetidas mientras que los 5 tokens NOUN únicamente aparecen 1 vez cada uno, por lo que se le restan apariciones en el texto y se está dando automáticamente más peso a las entidades nombradas.

Por ello, vamos a ejecutar el mismo algoritmo que el caso anterior con la única diferencia que se va a devolver una lista de términos sin repetición.

In [ ]:
@register_case
def case_11(f_):
    """ Con entidades nombradas filtradas y mas comunes eliminando duplicados. """
    
    #Devolvemos
    return set(case_10(f_))

In [ ]:
print('Score: ', evaluate(cases_11))

## 2.12 TF-idf

Por último podemos probar a ejecutar estos mismos casos pero usando en vez de TD como médida, TF-idf.

In [ ]:
df_tf_idf = evaluate_all(measure=TF_idf)

display('df_tf_idf)

<a id='conclusions'></a>
# 3 Conclusiones

CONCLUSIONES

In [3]:
df_tf = evaluate_all()

df_tf.plot(kind='bar')

KeyError: 'id'

In [ ]:
df_tf.plot(kind='bar')

In [ ]:
df_tf_idf.plot(kind='bar')

<a id='Anexo I'></a>
# I Funciones auxiliares

Vamos a definir los métodos que vamos a utilizar para clusterizar los documentos:

In [ ]:
# Function to create a TF vector for one document. For each of
# our unique words, we have a feature which is the tf for that word
# in the current document
def TF(document, unique_terms, collection):
    word_tf = []
    for word in unique_terms:
        word_tf.append(collection.tf(word, document))
    return word_tf

def cluster_texts(texts, cluster_number, distance):
    #Load the list of texts into a TextCollection object.
    collection = nltk.TextCollection(texts)
    print("Creando collecion de %d terminos" % len(collection))

    #get a list of unique terms
    unique_terms = list(set(collection))
    print("Terminos unicos encontrados: ", len(unique_terms))

    ### And here we actually call the function and create our array of vectors.
    vectors = [numpy.array(TF(f,unique_terms, collection)) for f in texts]

    # initialize the clusterer
    clusterer = AgglomerativeClustering(n_clusters=cluster_number,
                                      linkage="average", affinity='cosine')
    clusters = clusterer.fit_predict(vectors)

    return clusters



Para gestionar los diferentes casos, hemos generado unos métodos y funciones de ayuda para simplemente implementar una función que reciba un fichero de entrada y devuelva la lista de términos que serán analizados por el algoritmo de clusterización.

In [ ]:
def timer_decorator(func):
    def func_wrapper(*args):
        start = time()
        res = func(*args)
        end = time()
        diff_time = end - start
        if (diff_time < 60):
            print("Tiempo total de ejecución {:.2f} segundos".format(diff_time))
        else:
            print("Tiempo total de ejecución {:.2f} minutos".format(diff_time/60))
        return res
    return func_wrapper


cases = list()


def register_case(func):
    cases.append(func)
    def func_wrapper(*args):
        return func(*args)
    return func_wrapper

@timer_decorator
def evaluate(func, corpus_dir="./corpus_text"):
    texts = []
    for path in sorted([f for f in os.listdir(corpus_dir)
                        if f.endswith(".txt")]):
        with open(os.path.join(corpus_dir, path), "r") as f_:
            tokens = func(f_)
            texts.append(nltk.Text(tokens))
    test = cluster_texts(texts, 5, "cosine")
    print("Clusters obtenidos: ", test)
    print("Clusters esperados: ", REFERENCE)
    return adjusted_rand_score(REFERENCE, test)


def bulk_evaluate(*funcs):
    print("Reference: " , ", ".join([str(r) for r in REFERENCE]))
    for func in funcs:
        print("Evaluando %s" % func.__name__)
        print("Puntuacion: ", evaluate(func))
